In [1]:
const name = 
const residual_alphas = []
const implicit = true;

In [2]:
import SparseArrays: sparse
import NBInclude:@nbinclude
@nbinclude("Alpha.ipynb");

## Training

In [3]:
function save_linear_model(alphas, implicit, outdir)
    _, β = regress(alphas, implicit)
    @info "alphas: $alphas"
    @info "coefficients: $β"
    write_params(Dict("β" => β, "alphas" => alphas), outdir)

    splits = []
    @showprogress for split in all_splits
        push!(splits, read_alpha(alphas, split, implicit))
    end
    splits = reduce(cat, splits)
    preds = sparse(splits.user, splits.item, splits.rating)

    function model(users, items)
        p = Array{eltype(preds)}(undef, length(users))
        Threads.@threads for i = 1:length(p)
            p[i] = preds[users[i], items[i]]
        end
        p
    end
    write_alpha(model, residual_alphas, implicit, outdir)
end;

In [4]:
save_linear_model(
    [
        "SimpleAverage"
        "SimpleExplicitBaseline"
        "UserItemBiases"
    ],
    false,
    "LinearExplicit",
);

Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.79 μs/it)it)
[ Info: 20220605 19:24:37 alphas: ["SimpleAverage", "SimpleExplicitBaseline", "UserItemBiases"]
[ Info: 20220605 19:24:38 coefficients: Float32[0.004006544, -0.032630056, 1.0311745]
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
[ Info: 20220605 19:25:21 validation loss: 1.8078513, β: Float32[1.0000083]
Progress: 100%|███████████████████████████| Time: 0:00:00 (38.09 ns/it)
[ Info: 20220605 19:25:43 training loss: 1.6750224, β: Float32[1.0000083]


In [5]:
save_linear_model(
    [
        "UniformBaseline"
        "PopularityBaseline"
    ],
    true,
    "LinearImplicit",
);

Progress: 100%|███████████████████████████| Time: 0:00:00 (46.82 ns/it)
[ Info: 20220605 19:27:25 alphas: ["UniformBaseline", "PopularityBaseline"]
[ Info: 20220605 19:27:25 coefficients: Float32[1.6116662f-6, 0.99999666, 1.6116632f-6]
Progress: 100%|█████████████████████████████████████████| Time: 0:04:23
[ Info: 20220605 19:33:06 validation loss: 7.475159, β: Float32[4.933168f-7, 0.9999995]
Progress: 100%|███████████████████████████| Time: 0:00:00 (39.06 ns/it)
[ Info: 20220605 19:33:40 training loss: 7.4384027, β: Float32[4.933168f-7, 0.9999995]


In [6]:
# SOTA: 6.4687146803286275